# **Modelo de Recomendação**
Nesse notebok será trabalhado no modelo de recomendação de produtos com dados de usuários de Olist, que foram selecionados e gerados a partir do notebook **analise_exploratoria_olist.ipnyb** (recomendo a leitura). Além de treinarmos o modelo para recomendação, vamos validar sua qualidade de previsão, testar dados absolutos e normalizados, e alguns modelos para entender qual seria a melhor opção para essa base de dados.

## **Resumo do Modelo**
AAAAAAAAAAAAAAA

In [7]:
# Bibliotecas Necessárias para o modelo de recomendação
import pandas as pd
pd.set_option('display.max_columns', None)
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, GridSearchCV 
from surprise import accuracy
from collections import defaultdict

In [12]:
dataset = pd.read_excel("df_produtos_avaliacoes.xlsx")
dataset.head()

,product_id,user_id,rating,user_mean,user_reviews_count,normalized_rating
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,4,4.5,2,-0.5
1,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,4,4.0,1,0.0
2,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,5,5.0,1,0.0
3,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,5,5.0,1,0.0
4,ad21c59c0840e6cb83a9ceb5573f8159,72632f0f9dd73dfee390c9b22eb56dd6,5,5.0,1,0.0


---
# **Treino e Teste**
AAA

## **Carecterísticas do Modelo**
A biblioteca Surprise fornece a possibilidade da configuração de hiper-parâmetros para encontrarmos de uma maneira mais simples a configuração ideal através da função **GridSerachCV**. A partir dessa função, podemos colocar um array de valores para os hiper-parâmetros **n_factors**, **lr_all** e **reg_all**.

**n_factors**: Trata sobre a consideração de carecrísticas não vísiveis dos usuários que não estão no modelo.
- Entre 10 à 30 características ocultas é aplicado para modelos mais simples e diminuindo a chance de overfitting do modelo.
- Acima de 100 é aplicado para modelo mais complexos e pode aumentar a possibilidade overfitting para bases de dados pequenas.

**lr_all**: Taxa de aprendizado do modelo, controla o tamanho dos passos que o modelo segue durante o treino. Esse parâmetro é o que "ajusta" os pesos a cada iteração.
- Aplicando um valor baixo, como 0.002, o treino do modelo é mais lento, porém é considerado mais estável.
- Aplicando um valor alto, como 0.01, o treino do modelo é mais rápido, porém pode oscilar.

**reg_all**: Esse parâmetro é o que "penaliza", em maior ou menor peso, a previsão de dados mais distoantes. Esse parâmetro auxília para evitar o overfitting do modelo.
- Aplicando um valor baixo, como 0.02, há pouca penalização, aumento a possibilidade de overfitting.
- Aplicando um valor alto, como 0.01, há mais penalização, podendo substimar relações reais.

In [13]:
reader = Reader(rating_scale=(1, 5)) #Definindo a escala das notas e a função de leitura da lib
data_abs = Dataset.load_from_df(dataset[['user_id', 'product_id', 'rating']], reader) # Nota absoluta
data_nor = Dataset.load_from_df(dataset[['user_id', 'product_id', 'normalized_rating']], reader) #Nota normalizada

In [15]:
#Definição dos hiper-parâmetros
param_grid = {
    'n_factors': [10, 30, 50, 125, 150],
    'lr_all': [0.005, 0.008, 0.009, 0.01, 0.02],
    'reg_all': [0.05, 0.075, 0.1, 0.15, 0.2]
}

#Definição das carecterísticas do modelo
gs = GridSearchCV(SVD, #Modelo
                  param_grid, #Hiper parâmetros
                  measures=['rmse', 'mae'], #Métricas de Avaliação
                  cv=5, #Qtd de recortes na bases de dados 
                  joblib_verbose=2) # Controle do nível de verbosidade

Modelos como SVD, SVD++, e BaselineOnly funcionam melhor quando os dados foram centrados em torno de médias.

O SVD, por exemplo, decompõe a matriz de interações — se as notas estiverem normalizadas, a decomposição converge melhor e capta padrões reais.


In [16]:
gs.fit(data_abs) #Treino com as notas absolutas

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   22.2s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:  1.6min
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:  4.4min


In [17]:
#Verificação dos melhores hiper-parâmetros e algumas métricas de avaliação
print('RMSE', gs.best_params['rmse'])
print(f"RMSE Score: {gs.best_score['rmse']:.4f}")
print('\nMAE', gs.best_params['mae'])
print(f"MAE Score: {gs.best_score['mae']:.4f}")

RMSE {'n_factors': 10, 'lr_all': 0.02, 'reg_all': 0.1}
RMSE Score: 1.2546

MAE {'n_factors': 10, 'lr_all': 0.02, 'reg_all': 0.05}
MAE Score: 0.9699


In [18]:
gs.fit(data_nor) #Treino com as notas normalizadas

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   21.8s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:  1.6min
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:  4.4min


In [19]:
#Verificação dos melhores hiper-parâmetros e algumas métricas de avaliação
print('RMSE', gs.best_params['rmse'])
print(f"RMSE Score: {gs.best_score['rmse']:.4f}")
print('\nMAE', gs.best_params['mae'])
print(f"MAE Score: {gs.best_score['mae']:.4f}")

RMSE {'n_factors': 10, 'lr_all': 0.005, 'reg_all': 0.05}
RMSE Score: 1.0137

MAE {'n_factors': 10, 'lr_all': 0.005, 'reg_all': 0.05}
MAE Score: 1.0048


-----
Após realizar o treino com vários valores para parâmetros, chegou o momento de verificar, com base no resultado do RMSE e o MAE, quais valores dos hiper parâmetros são os mais ideais para gerar um modelo de recomendação de maior qualidade de previsão.

Antes de verificar os melhores valores para os hiper parâmetros, vamos relembrar o que significa o RMSE e o MAE que vamos usar como base para essa escolha:

<span style="font-size:20px">**Root Mean Squared Error (RMSE)**</span>

Essa métrica mede o erro médio ao quadrado entre o valor previsto e o valor real, auxilia no entendimento de quando o modelo pode cometer erros mais altos de previsão.

<span style="font-size:20px">**Mean Absolute Error (MAE)**</span>

Essa métrica mede a diferença absoluta média entre o valor previsto e o valor real, auxilia no entendimento da qualidade do modelo em prever os valores reais.


**Interpretação**

Para esse caso, que temos notas de produtos entre 1 e 5, poderiamos interpretar o RMSE e o MAE da seguinte forma:

Abaixo 0.7 - Excelente | Entre 0.7 e 0.9 - Bom | Entre 0.9 e 1.1 - Ok | Acima de 1.1 - Ruim

## **Treino, Teste e Validação**

In [19]:
# Divisão dos dados
trainset, testset = train_test_split(data, test_size=0.25)

# Modelo com os melhores parâmetros
best_model = gs.best_estimator['rmse']
best_model.fit(trainset)

# Avaliação no conjunto de teste
predictions = best_model.test(testset)
print("RMSE no teste:", accuracy.rmse(predictions))
print("MAE no teste:", accuracy.mae(predictions))

RMSE: nan
RMSE no teste: nan
MAE:  nan
MAE no teste: nan


In [20]:
def precision_recall_f1_at_k(predictions, k=3, threshold=4.0):
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
        
    precisions, recalls, f1s = [], [], []
    for est_true in user_est_true.values():
        est_true.sort(key=lambda x: x[0], reverse=True)
        top_k = est_true[:k]
        tp = sum((true_r >= threshold) for _, true_r in top_k)
        fp = k - tp
        fn = sum((true_r >= threshold) for _, true_r in est_true[k:])
        
        precision = tp / (tp + fp) if (tp + fp) else 0
        recall = tp / (tp + fn) if (tp + fn) else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0
        
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
    
    return {
        'Precision': sum(precisions) / len(precisions),
        'Recall': sum(recalls) / len(recalls),
        'F1': sum(f1s) / len(f1s)
    }


In [21]:
metrics = precision_recall_f1_at_k(predictions, k=3)
print("Métricas Top-N:", metrics)

Métricas Top-N: {'Precision': 0.265147721582319, 'Recall': 0.7871877260334947, 'F1': 0.395999554887888}


**Treino com todos os dados**

In [22]:
# Treinar com 100% dos dados
trainset_full = data.build_full_trainset()
best_model.fit(trainset_full)

In [23]:
# Treinar com 100% dos dados
trainset_full = data.build_full_trainset()
best_model.fit(trainset_full)

# Recomendação para usuário 1: itens que ele ainda não avaliou
user_id = "345ecd01c38d18a9036ed96c73b8d066"
all_items = set(iid for (_, iid, _) in trainset_full.all_ratings())
rated_items = set(j for (j, _) in trainset_full.ur[trainset_full.to_inner_uid(user_id)])
unseen_items = all_items - rated_items

# Predizer e ordenar por melhor nota prevista
recommendations = [
    (iid, best_model.predict(user_id, iid).est) for iid in unseen_items
]
recommendations.sort(key=lambda x: x[1], reverse=True)

# Top 3 recomendações
print(f"Top 3 recomendações para o usuário {user_id}:")
for iid, score in recommendations[:3]:
    print(f"Item {iid} - Nota prevista: {score:.2f}")


Top 3 recomendações para o usuário 345ecd01c38d18a9036ed96c73b8d066:
Item 0 - Nota prevista: 5.00
Item 1 - Nota prevista: 5.00
Item 2 - Nota prevista: 5.00


In [24]:
# Função para recomendar produtos a um usuário específico
def recomendar_produtos(user_id, df, model, n_recommendations=3):
    """Gera recomendações de produtos para um usuário."""
    
    # Obtém todos os produtos únicos
    all_products = df['product_id'].unique()
    
    # Obtém produtos já avaliados pelo usuário
    rated_products = df[df['user_id'] == user_id]['product_id'].values
    
    # Filtra apenas os produtos que o usuário ainda não avaliou
    products_to_predict = [p for p in all_products if p not in rated_products]
    
    # Faz previsões para esses produtos
    predictions = [(p, model.predict(user_id, p).est) for p in products_to_predict]
    
    # Ordena por nota prevista (maior para menor)
    predictions.sort(key=lambda x: x[1], reverse=True)
    
    return predictions[:n_recommendations]

# Exemplo: Recomendação para o usuário 1
user_id = 1
recommendations = recomendar_produtos(user_id, df, model)

print(f"\nRecomendações para o usuário {user_id}:")
for product, rating in recommendations:
    print(f"Produto {product} - Nota prevista: {rating:.2f}")

NameError: name 'df' is not defined

# Recomendação de Produtos

**Separação dos dados para recomendação**

In [ ]:
df_ultima_compra = dataframe.groupby("customer_unique_id").agg({'order_purchase_timestamp':max})
df_ultima_compra.reset_index(inplace=True)

In [ ]:
df_ultima_compra['order_purchase_timestamp'] = pd.to_datetime(df_ultima_compra['order_purchase_timestamp'])
data_referencia = pd.to_datetime('2018-09-10')
df_ultima_compra['dias_desde_da_ultima_compra'] = (data_referencia - df_ultima_compra['order_purchase_timestamp']).dt.days

In [ ]:
df_30_90_dias = df_ultima_compra[(df_ultima_compra['dias_desde_da_ultima_compra'] >= 30) & (df_ultima_compra['dias_desde_da_ultima_compra'] <= 90)]['customer_unique_id']
df_30_90_dias = df_30_90_dias.to_list()

TESTE